# The purpose of this notebook

In discussion with Michael, I mentioned that we support our use of a linear model of the suppression data by showing that it has better mean and median R^2 than a linear fit to the log-logged data. However, both box plots of fits showed a group of curves (each being one eye of one patient viewing the target under a certain surround/presentation condition etc) that were very badly fit by either model, like near zero and enough to be outliers on a boxplot. So, we thought maybe we'd remove them and see if that changes the results. That's what I'm attempting to do here; The immediately previous notebook, whose graphs are in redo-201901, are the comparison (these are in redo-2901902-exclude_bad_fits)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import string

import numpy as np
np.set_printoptions(precision=3)

import pandas as pd
import scipy.stats as st
import statsmodels.stats.multitest as mt

import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns

from scipy import stats

import suppression as s
import utils

In [ ]:
pd.__version__

In [ ]:
sns.__version__

In [ ]:
sns

In [ ]:
gaba_fn = 'gaba_data_2019.txt'
supp_fn = 'supp_data_individual_20170427.txt'

In [ ]:
sdf = utils.load_psychophys(supp_fn)
gaba_col = 'mean_occ_all' #'motor' # or 'occ_binoc', 'mean_occ_all', 'motor'
gdf = utils.load_gaba(gaba_fn, gaba_col)
sdf.head()

In [ ]:
gdf.Population = gdf.Population.astype('category')
gdf.Population.cat.categories # 0 AMB 1 CON
gdf.Population = gdf.Population.cat.rename_categories(['Persons with Amblyopia', 'Normally-sighted persons'])

In [ ]:
demos = pd.read_csv('demos.csv', lineterminator="\r")
# 1 = amb, 0 = control
subs_to_swap_eyes = demos[demos.swapNDE_EY==1].initials.unique() # subjects whose NDE/DE assignment is wrong in sdf
print(subs_to_swap_eyes)
demos

In [ ]:
demos[demos.initials=='nl']

### Set variables used for graphing

In [ ]:
colors_amb = ["#3274a1","#72b4e1"]
colors_con = ["#e1812c", "#ffc68c"]
colors4 = colors_amb + colors_con
traces4 = ['Amblyope-De', 'Amblyope-Nde', 'Control-De', 'Control-Nde']
traces_graph4 = [f"Persons with\nAmblyopia, DE", f"Persons with\nAmblyopia, NDE", \
                 f"Normally-sighted\npersons, DE", f"Normally-sighted\npersons, NDE"]
plot_dir = f"plots/redo-202001-{gaba_col}-fixeyes"

## Analyze tasks separately (before subsetting to include common subjects)##

In [ ]:
pp_subjs = np.unique(sdf.Subject)
n_pp_subjs = len(pp_subjs)
gaba_subjs = np.unique(gdf.subjName)
n_gaba_subjs = len(gaba_subjs)
print(f"Psychophysics subjects (n={n_pp_subjs}):\n", pp_subjs)
print(f"GABA subjects (n={n_gaba_subjs}):\n", gaba_subjs)

# GABA only analyses

### GABA t-test, CON v AMB

In [ ]:
gdf

#### Remove AM and TT based on discussions with Kelly 12/2019.

In [ ]:
gdf_reduced = gdf[(gdf.subjName != 'am') & (gdf.subjName !='tt')]
pop_group_reduced = gdf_reduced.groupby("Population")
pop_group_reduced.describe()

In [ ]:
#print(*pop_group_reduced['GABA'])
gaba_per_group = [col for col_name, col in pop_group_reduced['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group, nan_policy='omit')
print(tstat, pval)

In [ ]:
pop_group = gdf.groupby("Population")
pop_group.describe(percentiles=[.5])

In [ ]:
gaba_per_group = [col for col_name, col in pop_group['GABA']]
(tstat, pval) = st.ttest_ind(*gaba_per_group, nan_policy='omit')
print(tstat, pval)

**Thus we find no significant difference in GABA levels between CON and AMB regardless of whether AM and TT are included. At this point we may as well proceed with gdf_reduced only.**

### GABA violin plot, all subjects

In [ ]:
gdf = gdf_reduced.copy()
gaba_df_immutable = gdf_reduced.copy()
gaba_df_immutable.groupby("Population").describe() # Shoud be 14 PWA due to AM, TT exclusion

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_diffs_n{n_gaba_subjs}_{gaba_col}.pdf") as pdf:
    #with sns.plotting_context(context=None, font_scale=1.3):
    sns.set_context(context="paper", font_scale=1.3)
    fig = plt.figure(figsize=(8,8))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    ax = sns.violinplot(y='GABA',x='Presentation',hue='Population',data=gaba_df_immutable,split=True,inner='stick',ax=ax,legend=False)
    ax.legend()
    ax.xaxis.set_visible(False)
    ax.set_ylabel('GABA:Creatine ratio')
    sns.despine(left=True, bottom=True, right=True)
    #ax.set_yticklabels([])
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')

### Note: the subject with the lowest GABA:Cr ('tt', .162) is not in the psychophysics data

# Select one psychophysical task's data #

In [ ]:
task = 'SS' # 'SS'
sdf = sdf[sdf['Task']==task]
df_to_model = sdf.copy() # make a deep copy

In [ ]:
df_to_model.head()

In [ ]:
n_pp_subjs_thistask = len(np.unique(df_to_model.Subject))
amb_subjs = np.unique(df_to_model[df_to_model["Population"]=="Amblyope"]["Subject"])
print(amb_subjs)
n_amb_subjs_thistask = len(amb_subjs)
print(f"There are {n_pp_subjs_thistask} subjects for Task {task}, of which {n_amb_subjs_thistask} are Amblyopes.")

### Verifying baselines based on KB feedback about fig R2

In [ ]:
onecond = df_to_model[(df_to_model['Presentation']=='nMono') & (df_to_model['Orientation']=='Iso')]

In [ ]:
cnde_subs = onecond[onecond['Trace']=='Control-Nde'].Subject.unique()

In [ ]:
cde_subs = onecond[onecond['Trace']=='Control-De'].Subject.unique()

In [ ]:
np.setdiff1d(cnde_subs, cde_subs)

In [ ]:
onecond.groupby(['Task','Orientation','Presentation','Population', 'Eye','Trace'])['BaselineThresh'].describe()

In [ ]:
thresh_noswap = onecond.groupby(['Task','Orientation','Presentation','Population', 'Eye','Trace',"Subject"])['BaselineThresh'].mean().reset_index()

In [ ]:
thresh_noswap.head()

In [ ]:
eye_counts = thresh_noswap['Subject'].value_counts().reset_index()

In [ ]:
eye_counts

In [ ]:
nounpaired = (eye_counts[eye_counts.Subject==2])['index'].unique()

In [ ]:
nounpaired

In [ ]:
thresh_noswap_nounpaired = thresh_noswap[thresh_noswap['Subject'].isin(nounpaired)]

In [ ]:
thresh_noswap.groupby(['Trace']).mean().reset_index()

In [ ]:
thresh_noswap_nounpaired.groupby(['Trace']).describe()

In [ ]:
sns.barplot(data=thresh_noswap, x="Trace", y="BaselineThresh")

In [ ]:
sns.swarmplot(data=onecond, x="Population", y="BaselineThresh", hue="Eye")

### This is where the NDE/DE should be switched based on KB findings (only affects Controls, luckily)

But as the above plot shows, there are many more in Control-Nde than Control-De

In [ ]:
def swap_eyevars(df, subs=subs_to_swap_eyes):
    #print(df, df['Subject'], len(df), sep='\n')
    if df['Subject'] in subs: # fix here
        print(df['Subject'], "SWAP!")
        if df['Eye'] == "De":
            df['Eye'] = "Nde"
            df['Trace'] = df['Trace'].replace('-De', '-Nde')
        else:
            df['Eye'] = "De"
            df['Trace'] = df['Trace'].replace('-Nde', '-De')
    return df

def fix_eyes(df):
    disp_cols = ['Subject','Eye','Trace','BaselineThresh']
    rows_to_change = df_to_model[df_to_model.Subject.isin(subs_to_swap_eyes)]
    print(rows_to_change[disp_cols])
    assert(np.all(rows_to_change.Population == 'Control')) # should only affect controls
    assert(np.all(rows_to_change.Trace.isin(['Control-De','Control-Nde']))) # these should be swapped along w/ Eye
    fixed = df.apply(swap_eyevars, axis=1)
    print(fixed[disp_cols])
    return fixed

df_to_model_fixeyes = fix_eyes(df_to_model)

### Toggle which version of the data to use, "fixed" eyes or not

In [ ]:
df_to_model = df_to_model_fixeyes

### GABA vs baseline contrast threshold, before excluding bad fits

In [ ]:
baseline_gaba_plot_df = df_to_model.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
simple_bg_df = baseline_gaba_plot_df.groupby(['Task','Orientation','Presentation','Population','Subject','Eye','Trace'])[['GABA','BaselineThresh']].agg(np.mean).reset_index()

In [ ]:
simple_bg_df = simple_bg_df.rename(columns={"BaselineThresh":"value"})
simple_bg_df['measure'] = "BaselineThresh"
simple_bg_df.head()

In [ ]:
len(simple_bg_df.Subject.unique())

In [ ]:
simple_bg_df.Trace = simple_bg_df.Trace.astype('category')
simple_bg_df.Trace.cat.reorder_categories(traces4, inplace=True)
simple_bg_df.Trace.cat.rename_categories(traces_graph4, inplace=True)

In [ ]:
simple_bg_df.groupby(['Task','Orientation','Presentation','Population', 'Eye', 'measure'])['value'].describe()

In [ ]:
bg_groups = simple_bg_df.groupby(['Task','Orientation','Presentation','measure'])
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_{gaba_col}_baselines_noexc.pdf") as pdf:
    sns.set_context(context="paper", font_scale=1.3)
    for gv, gr in bg_groups:
        print(gv)
        g = s.gaba_vs_psychophys_plot(gv, gr,
                    legend_box=[0.9, 0.60, 0.1, 0.15], log=True, ylim=(1, 30),
                    col="Population", hue="Trace",
                    palette=colors4,
                    n_boot=5000, legend_img=False, markers=['o','s','o','s'])
        #g.fig.suptitle(f"{gv}", fontsize=10, y=0.999)
        g.set_titles("")
        g.fig.subplots_adjust(left=.08, wspace=.1, right=.82)
        pdf.savefig(g.fig)
        plt.close('all')

# Preparation for Modeling

In [ ]:
df_to_model.head() # note the first rows, they will tell if fixed - ai/Nde/6.9 -> ai/De/6.9 

### Begin grouping data into conditions to model Subject's ThreshElev as a function of logRelContrast #

In [ ]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace'] # One condition
pp_gvars_base = pp_gvars + ['BaselineThresh']

groups_with_baseline = df_to_model.groupby(pp_gvars_base)

# Check if there are any conditions with only two data points
for gv, gr in groups_with_baseline:
    if len(gr)<=2:
        print(gv, gr)

### BaselineThresh analysis before we exclude bad fits; since this is observed not modeled its ok

In [ ]:
print(pp_gvars_base)
pp_gvars_base_agg = [v for v in pp_gvars if v != 'Subject']

In [ ]:
pp_gvars_base_agg

In [ ]:
for gv, g in df_to_model.groupby(pp_gvars_base_agg):
    print(gv, len(np.unique(g['BaselineThresh'])))

In [ ]:
thresh_swap = df_to_model.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'Subject','Eye', 'Trace'])['BaselineThresh'].mean().reset_index()
thresh_swap = thresh_swap[(thresh_swap.Orientation=="Iso")&(thresh_swap.Presentation=='nDicho')]
thresh_swap.head()

In [ ]:
eye_counts_swap = thresh_swap.Subject.value_counts()
subs_no_unpaired = eye_counts[eye_counts_swap.reset_index()['Subject']==2]['index'].unique()
print(subs_no_unpaired, len(subs_no_unpaired))

In [ ]:
thresh_swap_nounpaired = thresh_swap[thresh_swap.Subject.isin(subs_no_unpaired)]

In [ ]:
pd.Series.value_counts(thresh_swap_nounpaired.Subject)

In [ ]:
thresh_swap_nounpaired.groupby(['Trace']).describe()

#### Calculate differences, ratios etc for each subject

In [ ]:
baseline_df_withinsubject = thresh_swap_nounpaired.groupby(['Task', 'Orientation', 'Presentation', 'Population',
                                        'Subject']).apply(utils.get_interocular_baseline_diff, 'BaselineThresh')
baseline_df_withinsubject[(baseline_df_withinsubject.Population=="Amblyope") & (baseline_df_withinsubject.Eye=="De")]

In [ ]:
ade_threshs = baseline_df_withinsubject[(baseline_df_withinsubject.Population=="Amblyope") & (baseline_df_withinsubject.Eye=="De")]

In [ ]:
baseline_df_reduced = baseline_df_withinsubject.groupby(pp_gvars_base_agg).apply(utils.describe_baselines).reset_index()

In [ ]:
baseline_df_reduced.head()

### Seaborn plots of baselines and ratios, just to see

In [ ]:
sns.barplot(data=baseline_df_withinsubject, x="Trace", y="BaselineThresh", palette=colors4)

In [ ]:
sns.barplot(data=baseline_df_withinsubject, x="Trace", y="BaselineDiff", palette=colors4)

In [ ]:
sns.barplot(data=baseline_df_withinsubject, x="Trace", y="BaselineRatio", palette=colors4)

### Begin within subject error bars, suggested by MAS 3/25

In [ ]:
baseline_df_withinsubject # our starting data frame

In [ ]:
baseline_df_withinsubject.groupby("Trace").mean()

Strategy could then be to normalize everyone within a trace (group) to the mean and then measure error?

### old pre-4/28 stuff follows

In [ ]:
baseline_df_withinsubject_diffs = baseline_df_withinsubject.groupby(['Population','Subject'])['BaselineDiff','BaselineRatio'].mean().reset_index()
baseline_df_withinsubject_diffs

###  What if we exclude subject 'ah' who looks like a real outlier?

In [ ]:
adiff = baseline_df_withinsubject_diffs[(baseline_df_withinsubject_diffs.Population=='Amblyope') &
                                        (baseline_df_withinsubject_diffs.Subject!="ah")]
cdiff = baseline_df_withinsubject_diffs[baseline_df_withinsubject_diffs.Population=='Control']

In [ ]:
print("diffs: NDE-DE")
print(st.ttest_1samp(adiff['BaselineDiff'], popmean=0), len(adiff['BaselineDiff']))
print(st.ttest_1samp(cdiff['BaselineDiff'], popmean=0), len(cdiff['BaselineDiff']))
print(st.ttest_ind(adiff['BaselineDiff'], cdiff['BaselineDiff']))
print("\nratios: NDE/DE")
print(st.ttest_1samp(adiff['BaselineRatio'], popmean=1))
print(st.ttest_1samp(cdiff['BaselineRatio'], popmean=1))
print(st.ttest_ind(adiff['BaselineRatio'], cdiff['BaselineRatio']))

### Ok based on conversation with Michael, 'ah' is excluded now

In [ ]:
baseline_df_withinsubject_diffs = baseline_df_withinsubject_diffs[baseline_df_withinsubject_diffs.Subject != 'ah']
df_to_model = df_to_model[df_to_model.Subject!='ah']

In [ ]:
baseline_df_withinsubject_diffs

In [ ]:
baseline_df_withinsubject_diffs.Population = baseline_df_withinsubject_diffs.Population.astype('category').cat.rename_categories(['Persons with Amblyopia', 'Normally-sighted persons'])
g = sns.boxplot(data=baseline_df_withinsubject_diffs, x='Population', y='BaselineDiff', hue='Population')
g.legend().remove()
#g.legend().set_title('')
g.set_xlabel('')
g.set_ylabel('Interocular difference (NDE-DE) in baseline\ncontrast discrimination thresholds (C%)')

In [ ]:
g = sns.boxplot(data=baseline_df_withinsubject_diffs, x='Population', y='BaselineRatio', hue='Population')
g.legend().remove()
g.set_xlabel('')
g.set_ylabel('Interocular ratio (NDE:DE) of baseline\ncontrast discrimination thresholds (C%)')
#g.set_yscale('log')
#g.set_ylim(0.5, 3)
#g.get_yaxis().set_major_locator(tick.LogLocator(subs=range(1, 10)))
g.get_yaxis().set_major_formatter(tick.ScalarFormatter())


In [ ]:
baseline_df_withinsubject_diffs.groupby(['Population']).describe()

### Make bar chart

In [ ]:
baseline_df_reduced

In [ ]:
baseline_plot_df = utils.make_baseline_df_to_plot(baseline_df_reduced, 'mean')

In [ ]:
baseline_plot_df.head()

In [ ]:
with s.PdfPages(f"{plot_dir}/{task}_baseline_diffs_group.pdf") as pdf:
    sns.set_context(context="paper", font_scale=1.3)
    fig = plt.figure(figsize=(8,6))  # create a figure object
    ax = fig.add_subplot(1, 1, 1)
    x_pos = np.arange(len(baseline_plot_df['Population']))
    plt.bar(x_pos, baseline_plot_df['mean'], data=baseline_plot_df, yerr='SEM', color=colors4)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(traces_graph4)
    ax.set_ylabel('Baseline Contrast Discrimination Threshold (C%)')
    ax.set_yscale('log')
    ax.set_ylim(1, 8)
    ax.get_yaxis().set_major_locator(tick.LogLocator(subs=range(1, 10)))
    ax.get_yaxis().set_major_formatter(tick.ScalarFormatter())
    sns.despine(left=True, bottom=True, right=True, top=True)
    plt.show(ax.figure)
    pdf.savefig(ax.figure)
    plt.close(ax.figure)
    plt.close('all')

## Actual fitting of model to contrast sensitivity curves
### Linear model using statsmodels

In [ ]:
lin_results = groups_with_baseline.apply(utils.linear_fit_params, 'RelMaskContrast', 'ThreshElev').reset_index()

In [ ]:
lin_results.head(n=16)

In [ ]:
sns.distplot(lin_results.rsquared, kde=False, rug=True)

In [ ]:
np.count_nonzero(lin_results.rsquared>.999999)

In [ ]:
lin_results.rsquared.mean()

In [ ]:
lin_results.boxplot(column='rsquared', by=['Orientation', 'Presentation'], grid=False, figsize=(16, 4))

In [ ]:
lin_results.boxplot(column='rsquared', by=['Orientation', 'Population', 'Presentation'], grid=False, figsize=(16, 4))

### Fit the log-logged data to see if that's better

In [ ]:
log_results = groups_with_baseline.apply(utils.linear_fit_params, 'logRelMaskContrast', 'logThreshElev').reset_index()

log_results.rsquared.mean()

In [ ]:
log_results.boxplot(column='rsquared', by=['Orientation', 'Population', 'Presentation'], grid=False, figsize=(16, 4))

## Conclusion: linear fits are overall better.

## Identify subjects with negative slope (for Jian)

In [ ]:
neg_slopes = lin_results[lin_results['slope']<0]
neg_slopes

In [ ]:
neg_slopes.Subject.unique()

## Exclude bad fits (new 2019-02-25)

In [ ]:
len(lin_results)

In [ ]:
lin_results_exc = lin_results.groupby(['Task', 'Population']).apply(utils.remove_outliers_halfvar).reset_index(drop=True)

In [ ]:
lin_results_exc.rsquared.min()

In [ ]:
lin_results_exc.Subject.value_counts().sum()

In [ ]:
231/256

In [ ]:
pp_subs_exc = lin_results_exc.Subject.unique()
print(f'There are {len(pp_subs_exc)} unique subjects who have at least one condition of data.')

## Actually use the linear model to predict thresholds

In [ ]:
lin_preds = groups_with_baseline.apply(utils.linear_fit_predictions, 'RelMaskContrast', 'ThreshElev').reset_index()

In [ ]:
lin_preds.columns

In [ ]:
lin_preds.head()

In [ ]:
plot_df = pd.merge(df_to_model, lin_preds, on=pp_gvars_base + ['RelMaskContrast'])

plot_df.head()

### Plot observed values and model fits

In [ ]:
example_fit_plot_df = plot_df[(plot_df.Subject=='gd') & 
                              (plot_df.Orientation=='Iso') & 
                              (plot_df.Presentation=='nDicho')]
example_fit_plot_df

In [ ]:
s.group_facet_plots(example_fit_plot_df, s.subject_fit_plot,
                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TOP.pdf",
                    ['Task','Orientation','Presentation',"Subject"], #each combo of this gets its own page
                    row='Presentation', col="Population",# facet rows and columns
                    x="RelMaskContrast", y="ThreshElev", # x, y
                    hue="Eye",yerr='ThreshElev_SE',fmt_obs='x',fmt_pred=':',Ycol="ThreshPred") 

In [ ]:
#s.group_facet_plots(plot_df, s.population_fit_plot,
#                    f"{plot_dir}/{task}_regressions_combinedplots_n{n_pp_subjs_thistask}_TO.pdf",
#                    ['Task','Orientation'], #each combo of this gets its own page
#                    row='Presentation',col='Eye',# facet rows and columns
#                    x="RelMaskContrast", y="ThreshElev", # x, y
#                    hue="Population",yerr='ThreshElev_SE',fmt_obs='.',fmt_pred='x:',Ycol="ThreshPred") 

## Now exclude the predictions for the bad fits

In [ ]:
lin_results.Orientation.value_counts()

In [ ]:
lin_results_exc.Orientation.value_counts()

In [ ]:
lin_results_exc.Population.value_counts()

In [ ]:
lin_results.Population.value_counts()

In [ ]:
comb_rsq_preds = pd.merge(lin_results_exc, lin_preds, on=pp_gvars_base)

In [ ]:
comb_rsq_preds.rsquared.min()

In [ ]:
comb_rsq_preds.head()

In [ ]:
comb_rsq_preds_noexc = pd.merge(lin_results, lin_preds, on=pp_gvars_base)

In [ ]:
comb_rsq_preds_noexc.rsquared.min()

In [ ]:
comb_rsq_preds_noexc.head()

### Pick an xvalue (RelMaskContrast) to evaluate models at

 * 2018-09-24: Abandoning Eunice's binning. Instead, try to figure out a good RelMaskContrast programatically.
   * Just looking at it via describe(), i'd say somewhere between 5 and 10 -- probably 6 (for SS) and 10 (for OS)
 * 2018-10-08: My previous approach was too subjective. Instead, evaluate model at various percentiles...
   * do this separately for Task, Orientation, Presentation (so pick 8 total numbers)
   * At this point it's easier to just use the statsmodels.ols functions maybe? The way it's currently done is a legacy that allows different models to be swapped in... which I hope to god is not going to be the direction we go in again.
   * nvm, used the lmfit solution since it returned a nicely formatted pfit df and is tested to work
   * Here I want to take the slope and y-int and calculate the model prediction at the specified percentiles above (0-1, increments of 0.2). So, first calculate the RelMC at each of those percentiles, then apply it like below.


In [ ]:
percentile_bins = np.linspace(0, 1, num=11)

In [ ]:
percentile_bins

#### Get the interpolated RelMaskContrasts for each regression line, i.e. the range of x-values

In [ ]:
relmc_pcts_df = comb_rsq_preds.groupby(pp_gvars_base)['RelMaskContrast'].describe(percentiles=percentile_bins)

In [ ]:
# fix stupid column naming from describe()
relmc_pcts_df.columns = [f"{int(float(col[:-1])):03d}" if col[-1]=="%" else col for col in relmc_pcts_df.columns]

In [ ]:
relmc_pcts_df.columns

In [ ]:
relmc_pcts_df = relmc_pcts_df.filter(regex='0|1')
relmc_pcts_df.head()

In [ ]:
relmc_pcts_df.columns

In [ ]:
relmc_pcts_df_melted = relmc_pcts_df.reset_index().melt(id_vars=pp_gvars_base, var_name='percentile', value_name='RelMaskContrast_pct')

In [ ]:
relmc_pcts_df_melted.head()

In [ ]:
predict_pcts_df = pd.merge(lin_results_exc, relmc_pcts_df_melted, on=pp_gvars_base)

In [ ]:
predict_pcts_df.head()

In [ ]:
predict_pcts_df['percentile'] = predict_pcts_df['percentile'].astype(int)
predict_pcts_df['RelMaskContrast_pct'] = predict_pcts_df['RelMaskContrast_pct'].astype(float)
predict_pcts_df['relmc_bin'] = (np.around(predict_pcts_df['RelMaskContrast_pct'])).astype(int)
predict_pcts_df['ThreshElev_pct'] = (predict_pcts_df['y_int'] + (predict_pcts_df['RelMaskContrast_pct']*predict_pcts_df['slope'])).astype('float')

In [ ]:
predict_pcts_df.head()

In [ ]:
predict_pcts_df.rsquared.min()

In [ ]:
gvars_test = ['Task','Orientation','Presentation','Population']
# equal_var=False makes it Welch's t-test, which does not assume the groups have equal variance
selected_bin_df = utils.find_pct_to_predict(predict_pcts_df, gvars_test,
                    'relmc_bin', 'ThreshElev_pct', test_func=st.ttest_ind, equal_var=False)

In [ ]:
g_TOP = selected_bin_df.groupby(['Task', 'Orientation', 'Presentation'])
def set_relmctopred_to_amb_val(g):
    ambs = g[g['Population']=='Amblyope']
    assert(np.all(ambs['RelMCToPred']==ambs['RelMCToPred'].iat[0]))
    assert(np.all(ambs['BinNumberToPred']==ambs['BinNumberToPred'].iat[0]))
    g['RelMCToPred'] = ambs['RelMCToPred'].iat[0]
    g['BinNumberToPred'] = ambs['BinNumberToPred'].iat[0]
    return g
selected_bin_df = g_TOP.apply(set_relmctopred_to_amb_val).reset_index()

In [ ]:
selected_bin_df.groupby(gvars_test)['RelMCToPred','ThreshElev_pct'].describe()

In [ ]:
selected_bin_df['ThreshPredCritical'] = selected_bin_df['y_int'] + selected_bin_df['slope'] * selected_bin_df['RelMCToPred']
selected_bin_df['ThreshPredCriticalUnnorm'] = selected_bin_df['ThreshPredCritical'] * selected_bin_df['BaselineThresh']

In [ ]:
selected_bin_df

In [ ]:
individual_plot_df = selected_bin_df[(selected_bin_df.Subject=='sd') & (selected_bin_df.Presentation=='nDicho') &
                (selected_bin_df.Orientation=='Iso')]

In [ ]:
sns.scatterplot(data=individual_plot_df, x="RelMaskContrast_pct", y="ThreshElev_pct", hue="Eye")

### Melt the result of the modeling into long format for plotting

In [ ]:
pfit_all_ppsub = pd.melt(selected_bin_df, id_vars=[*pp_gvars, 'rsquared'],
                    value_vars=['BaselineThresh', 'y_int', 'slope', 'ThreshPredCritical', 'ThreshPredCriticalUnnorm'],
                    var_name='measure')
pfit_all_ppsub = pfit_all_ppsub[pfit_all_ppsub.Subject != 'ah']
pfit_all_ppsub.head()

In [ ]:
pp_stats = pfit_all_ppsub[(pfit_all_ppsub.measure=="ThreshPredCritical")].drop_duplicates()

In [ ]:
pp_slopes = pfit_all_ppsub[pfit_all_ppsub.measure=="slope"].drop_duplicates()

In [ ]:
pp_stats.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'Eye', 'Trace','Subject','measure'])['value'].describe(percentiles=[.5])

In [ ]:
def test_suppression_diffs(g):
    ndes = np.unique(g[g.Eye=='Nde']['value'])
    des = np.unique(g[g.Eye=='De']['value'])
    #g.hist()
    print(len(ndes), ' ', len(des))
    print(st.ttest_ind(ndes, des))
    return st.ttest_ind(ndes, des)

gs = pp_stats.groupby(['Task', 'Orientation', 'Presentation', 'Population', 'measure'])
for gv, g in gs:
    if gv[-1] == 'ThreshPredCritical':
        print(gv)
        test_suppression_diffs(g)
        #test_suppression_diffs_withinsubject(g)

## Subset to include only (GABA and psychophyics) subjects

In [ ]:
gaba_and_pp_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
n_gaba_and_pp_subjs = len(gaba_and_pp_subjs)

In [ ]:
sdf = sdf[sdf.Subject.isin(gaba_and_pp_subjs)] # only subjects who did _the current_ pp task and GABA
gaba_and_pp_subjs_thistask = np.unique(sdf.Subject)
n_gaba_and_pp_subjs_thistask = len(gaba_and_pp_subjs_thistask)
print(f"Of the {n_gaba_and_pp_subjs} subjects with both GABA and psychophysics data, {n_gaba_and_pp_subjs_thistask} have both for task {task}.\n{gaba_and_pp_subjs_thistask}")

### Remove subjects we don't have data on both GABA/PP for

In [ ]:
gdf = gdf[gdf.subjName.isin(sdf.Subject)] # only subjects who did both tasks
amb_subjs = (gdf[gdf.Population=='Persons with Amblyopia'])
print(f'Of the {len(gdf)} subjects with GABA and {task} data, {len(amb_subjs)} are Amblyopes.')
n_this_task = len(gdf)

In [ ]:
stats_thistask = lin_results_exc[lin_results_exc.Subject.isin(gaba_and_pp_subjs_thistask)].groupby(['Subject'])['rsquared'].describe()

In [ ]:
stats_thistask

In [ ]:
n_gaba_and_pp_subjs_thistask * 8 - stats_thistask['count'].sum()

## Combine Psychophysics and GABA below

In [ ]:
#Grab the GABA measure for each subject and append it to each observation for easy plotting
comb = pfit_all_ppsub.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])
comb.drop_duplicates(inplace=True)

#subset to include only those subjects with GABA data
comb_gabappsub = comb[~np.isnan(comb['GABA'])]
print(len(comb), len(comb_gabappsub))


In [ ]:
comb_gabappsub.Subject.unique()

### Do Spearman's R

In [ ]:
spearman_df = comb_gabappsub[(comb_gabappsub.measure == 'BaselineThresh') | 
                             (comb_gabappsub.measure == 'ThreshPredCritical')]

In [ ]:
spearman_df.Subject.unique()

In [ ]:
spearman_df.Trace = spearman_df.Trace.astype('category')
foo_temp = spearman_df.Trace.copy()
spearman_df.Trace.cat.reorder_categories(traces4, inplace=True)
assert(foo_temp.equals(spearman_df.Trace))
spearman_df.Trace.cat.rename_categories(traces_graph4, inplace=True)

In [ ]:
plot_groups = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure','Eye','Trace'])
for gv, gr in plot_groups:
    #if 'nDicho' in gv:
    print(gv)
    print(stats.spearmanr(gr.GABA, gr.value))

In [ ]:
#graphs!
colors_a = ["#3274a1","#72b4e1"]
colors_c = ["#e1812c", "#ffc68c"]
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}.pdf") as pdf:
    plot_groups = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure'])
    plot_groups_eacheye = spearman_df.groupby(['Task','Orientation','Presentation','Population','measure','Eye'])
    for gv, gr in plot_groups:
        #if gv[2] == 'nDicho':
        print(gv, np.all(np.isnan(gr['value'])), len(gr['value']))
        pal = [colors_a if 'Amblyope' in gv else colors_c]
        g2 = s.gaba_vs_psychophys_plot_2line_nofacet(gv, gr, palette=pal[0], aspect=1.2)
        pdf.savefig(g2.fig)

plt.close('all')

In [ ]:
# with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}_ind.pdf") as pdf:
#     for gv, gr in plot_groups:
#         print(gv)
#         g = s.gaba_vs_psychophys_plot(gv, gr, hue="Eye",
#                     palette=dict(De="g", Nde="m"),
#                     n_boot=1000, height=8, aspect=1, legend_out=True, truncate=True)
#         g.fig.suptitle(f"{gv} unweighted", fontsize=10, y=0.999)
#         pdf.savefig(g.fig)
#         plt.close('all')

In [ ]:
facet_groups = spearman_df.groupby(['Task','Orientation','Presentation','measure'])
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}_facet.pdf") as pdf:
    for gv, gr in facet_groups:
        print(gv)
        g = s.gaba_vs_psychophys_plot(gv, gr, 
                    legend_box=[0.9, 0.60, 0.1, 0.15],
                    legend_img = False,
                    log = False,
                    col="Population", hue="Trace",
                    palette=colors4,
                    n_boot=1000,
                    markers=['o','s','o','s'])#, legend=False)
        g.set_titles("")
        g.fig.subplots_adjust(left=.08, right=.82, wspace=0.1)
        pdf.savefig(g.fig)
        plt.close('all')

In [ ]:
facet_groups2 = spearman_df.groupby(['Task','Orientation','measure'])
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_{gaba_col}_facet_joint.pdf") as pdf:
    for gv, gr in facet_groups2:
        print(gv)
        g = s.gaba_vs_psychophys_plot(gv, gr, 
                    legend_box=[0.9, 0.60, 0.1, 0.15],
                    legend_img = False,
                    log = True,
                    #ylim = (1, 50),
                    row="Presentation", col="Population", hue="Trace",
                    palette=colors4,
                    n_boot=5000,
                    markers=['o','s','o','s'])#, legend=False)
        g.set_titles("")
        g.fig.subplots_adjust(left=.08, right=.82, wspace=0.1)
        pdf.savefig(g.fig)
        plt.close('all')

In [ ]:
spearman_supp_df = spearman_df[spearman_df.measure=="ThreshPredCritical"]
supp_facet_groups = spearman_supp_df.groupby(['Task','Orientation','Presentation','measure'])

In [ ]:
rdiffs_nores = supp_facet_groups.apply(utils.compare_rs, n_boot=1000, resample=False).reset_index()\
            .rename(columns={"level_4":"iteration"})

In [ ]:
# checking if the .015 p-val for SS/Iso/Dicho/AMB/Nde/ThreshPredCritical survives multicomp
pvals = np.array([0.0, 0.015, 0.08, 0.1, 0.15, 0.17, 0.2, 0.22, 0.25, 0.3, 0.33, 0.37, 0.4, 0.44, 0.47, 0.5])
print(pvals)
mt.multipletests(pvals, 0.05, 'holm')

In [ ]:
rdiffs = rdiffs_nores

In [ ]:
rdiffs_supp = rdiffs[rdiffs['measure']=="ThreshPredCritical"]
rdiffs_supp.head()

In [ ]:
rdiffs_supp.boxplot(column='amb_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
# Rho(GABA vs ThreshElev(NDE)) - Rho(GABA vs ThreshElev(DE)) for Amblyopes only, by condition
rdiffs_supp.hist(column='amb_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
rdiffs_supp.boxplot(column='con_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
rdiffs_supp.boxplot(column='pop_rdiff', by=['Task','Orientation','Presentation','measure'],
               grid=False, figsize=(16, 4))

In [ ]:
comb_gabappsub.head()

In [ ]:
spearman_df.head()

In [ ]:
spearman_df.measure.unique()

In [ ]:
kelly_file = f"{plot_dir}/{task}_data_frame.csv"
spearman_df.to_csv(kelly_file)

### Orientation Selective Suppression

In [ ]:
oss_gvars = ["Task", "Presentation", "Population", "Subject", "Eye", "Trace",
             "measure", "GABA"]
oss_gvars_combeyes = ["Task", "Presentation", "Population", "Subject", 
             "measure", "GABA"]

In [ ]:
for gv, g in spearman_df[spearman_df.measure=='ThreshPredCritical'].groupby(oss_gvars):
    print(gv, g.Orientation.unique())

In [ ]:
spearman_df[spearman_df.measure=='ThreshPredCritical'].groupby(['Task', 'Orientation', 'Presentation', 'Population','Eye'])['value'].describe()

In [ ]:
spearman_df[spearman_df.measure=='ThreshPredCritical'].groupby(['Task', 'Presentation', 'Population','Eye'])['Subject'].describe()

In [ ]:
oss_df = spearman_df[spearman_df.measure=='ThreshPredCritical'].groupby(oss_gvars).apply(utils.calculate_orientation_selective_suppression).reset_index()

In [ ]:
print(np.count_nonzero(np.isnan(oss_df.value)), len(oss_df.value))

In [ ]:
oss_df[~np.isnan(oss_df.value)]

In [ ]:
oss_df.boxplot(column='value', by=['Task','Presentation','Population','Eye'],
               grid=False, figsize=(16, 4))

In [ ]:
oss_df.groupby(['Task', 'Presentation', 'Population','Eye'])['value'].describe()

In [ ]:
for gv, g in oss_df.groupby(['Task', 'Presentation', 'Population','Eye']):
    print(gv)#, g, sep='\n')
    print(st.ttest_1samp(g['value'], popmean=1, nan_policy='omit'))

In [ ]:
def tt_pval(df):
    ttr = st.ttest_1samp(df['value'], popmean=1, nan_policy='omit')
    pval = ttr.pvalue
    return pd.Series(pval, ['pvalue'])

### OSS t-tests and results here

In [ ]:
for gv, g in oss_df.groupby(['Task', 'Presentation', 'Population']):
    print(gv, np.unique(g['Subject']))#, g, sep='\n')
    print(st.ttest_1samp(g['value'], popmean=1, nan_policy='omit'))

In [ ]:
oss_df.groupby(['Task', 'Presentation', 'Population'])['value'].describe()

In [ ]:
.345/(23**.5) # SEM of Control/Dicho (stdev from ^^^ / sqrt(n))

In [ ]:
oss_dicho = oss_df[oss_df.Presentation=="nDicho"]['value']
oss_mono = oss_df[oss_df.Presentation=="nMono"]['value']

In [ ]:
oss_dicho[~np.isnan(oss_dicho)]

In [ ]:
st.ttest_ind(oss_dicho, oss_mono, nan_policy='omit', equal_var=False)

In [ ]:
def oss_mean_combeyes(df, **kwargs):
    if len(df.Eye.unique())==2:
        v1 = df[df.Eye=='Nde']['value'].iloc[0]
        v2 = df[df.Eye=='De']['value'].iloc[0]
        oss_mean_combeyes = np.mean([v1, v2])
    else:
        oss_mean_combeyes = np.nan
    print(f"OSS mean across eyes: {oss_mean_combeyes}")
    return pd.Series(oss_mean_combeyes, ['value'])

In [ ]:
for gv, g in oss_df.groupby(oss_gvars_combeyes):
    print(gv, g, sep="\n")

In [ ]:
oss_df_combeyes = oss_df.groupby(oss_gvars_combeyes).apply(oss_mean_combeyes).reset_index()

In [ ]:
oss_df_combeyes

In [ ]:
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_oss.pdf") as pdf:
    temp_df = oss_df.copy()
    temp_df['Eye'] = temp_df['Eye'].astype('category')
    plot_groups = temp_df.groupby(['Task', 'Presentation', 'Population','measure'])
    for gv, gr in plot_groups:
        #if "BaselineThresh" in gv: continue
        if "Amblyope" in gv:
            pal = colors_a
        elif "Control" in gv:
            pal = colors_c
        else:
            print('Error! neither amb nor con!')
        print(gv, np.all(np.isnan(gr['value'])))
        g2 = s.gaba_vs_psychophys_plot_2line_nofacet(gv, gr, palette=pal)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
# with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_n{n_this_task}_oss_combeyes.pdf") as pdf:
#     temp_df = oss_df_combeyes.copy()
#     plot_groups = temp_df.groupby(['Task', 'Presentation', 'Population','measure'])
#     for gv, gr in plot_groups:
#         #if "BaselineThresh" in gv: continue
#         if "Amblyope" in gv:
#             pal = colors_a
#             print(gv, np.all(np.isnan(gr['value'])))
#         elif "Control" in gv:
#             pal = colors_c
#             print(gv, np.all(np.isnan(gr['value'])),
#                   stats.spearmanr(gr.GABA, gr.value, nan_policy='omit'), sep='\n')
#         else:
#             print('Error! neither amb nor con!')
#         g2 = s.oss_plot_2eye(gv, gr, palette=pal)
#         pdf.savefig(g2.fig)
        
#     plt.close('all')

### Combine measures across the two eyes

 * Does it make sense to combine all measures across both eyes (i.e. by subtracting?) For example, ThreshElev is in units of baseline, and the baseline varies by eye. So perhaps only a few measures should be combined -- say, slope/yint, ThreshPredCriticalUnnorm. 

In [ ]:
measures = comb_gabappsub[comb_gabappsub["measure"].isin(["BaselineThresh","ThreshPredCritical"])]

In [ ]:
np.unique(measures.measure)

In [ ]:
paired_obs = measures.groupby(['Task', 'Orientation', 'Population', 'Presentation', 'Subject', 'measure'])

def get_eyediff_value(g):
    if len(g)==2: # this will exclude paired observations where there was no data for one eye
        value_diff = g[g['Eye']=='Nde'].value.iat[0] - g[g['Eye']=='De'].value.iat[0]
        #print(g.name, value_diff)
        return pd.Series([value_diff], ['Nde-De'])
    else:
        print(f"Skipping because one eye is missing...")

In [ ]:
obs_diff = paired_obs.apply(get_eyediff_value).reset_index()

In [ ]:
obs_diff[obs_diff.Subject=='em']

In [ ]:
comb_botheyes = obs_diff.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [ ]:
comb_botheyes

In [ ]:
print(len(np.unique(comb_botheyes.Subject)))

In [ ]:
test_groups = comb_botheyes.groupby(['Task','Orientation','Presentation','Population','measure'])
for gv, gr in test_groups:
    #print(gr.head())
    if gv[-1]=="ThreshPredCritical":
        print(gv)
        print(stats.spearmanr(gr.GABA, gr['Nde-De']))

In [ ]:
comb_botheyes.Population = comb_botheyes.Population.astype('category')
comb_botheyes.Population = comb_botheyes.Population.cat.rename_categories(['Persons with Amblyopia', 'Normally-sighted persons'])

In [ ]:
#graphs!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv)
        g2 = s.gaba_vs_psychophys_plot_2line_2eye(gv, gr)
        pdf.savefig(g2.fig)
        
    plt.close('all')

In [ ]:
# more graphs for presentation!
with s.PdfPages(f"{plot_dir}/gaba_vs_{task}_combeyes_n{n_this_task}_poster.pdf") as pdf:
    plot_groups = comb_botheyes.groupby(['Task','Orientation','Presentation','measure'])
    for gv, gr in plot_groups:
        if gv[-2] in ["nDicho"]: # use this line to exclude measures we don't want
            print(gv)
            g2 = s.gaba_vs_psychophys_plot_2line_2eye_nofacet(gv, gr, hue="Population", height=5, aspect=1.2, legend=False)
            #print(g2.axes)
            pdf.savefig(g2.fig)
        
    plt.close('all')

## Demographic stuff from KB

In [ ]:
amb_data = demos[demos['group']==1]
amb_data['motorGABA'] = pd.to_numeric(amb_data['motorGABA'].str.strip(), errors='coerce')
amb_data[["numID","initials","labelGroup","labelNDE","swapNDE_EY","acuityDE","acuityNDE",
          "iadLogMAR","occGABA","motorGABA"]]

In [ ]:
#tt_eyes = st.ttest_ind(amb_data['acuityDE'], amb_data['acuityNDE'], nan_policy='omit') # are the eyes different in acuity?
#print(tt_eyes)
#print("occ gaba vs acuityDE: ", st.spearmanr(amb_data['occGABA'], amb_data['acuityDE'], nan_policy='omit'))
#print("occ gaba vs acuityNDE: ", st.spearmanr(amb_data['occGABA'], amb_data['acuityNDE'], nan_policy='omit'))
print("occ gaba vs IAD: ", st.spearmanr(amb_data['occGABA'], amb_data['iadLogMAR']))
print("motor gaba vs IAD: ", st.spearmanr(amb_data['motorGABA'], amb_data['iadLogMAR'], nan_policy='omit'))

In [ ]:
print("occ gaba vs interocular acuity difference: ", st.pearsonr(amb_data['occGABA'], amb_data['iadLogMAR']))
print("motor gaba vs interocular acuity difference: ", np.corrcoef(amb_data['motorGABA'], amb_data['iadLogMAR'])[0,1])
#print("motor gaba vs interocular acuity difference: ", st.pearsonr(amb_data['motorGABA'], amb_data['iadLogMAR']))

In [ ]:
print(amb_data['occGABA'], amb_data["motorGABA"], sep='\n')

In [ ]:
has_motor = amb_data[~np.isnan(amb_data['motorGABA'])]

In [ ]:
len(amb_data['occGABA']), len(has_motor['motorGABA'])

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_xlim(0.18, 0.23)
ax.set_ylim(0, 1.4)

sns.regplot(data=amb_data, x='occGABA', y='iadLogMAR', color='blue', marker='o', ax=ax, label='Occipital cortex')
r, p = st.pearsonr(amb_data['occGABA'], amb_data['iadLogMAR'])
ax.text(.35, 0.88, f"r={r:.3f}, p={p:.3f}", transform=ax.transAxes, fontdict={'color': 'blue'}, horizontalalignment='center')

sns.regplot(data=amb_data, x='motorGABA', y='iadLogMAR', color='grey', marker='x', ax=ax, label='Motor cortex')
r, p = st.pearsonr(has_motor['motorGABA'], has_motor['iadLogMAR'])
ax.text(.35, 0.82, f"r={r:.3f}, p={p:.3f}", transform=ax.transAxes, fontdict={'color': 'grey'}, horizontalalignment='center')

ax.legend()
ax.set_xlabel("GABA:Creatine Ratio")
ax.set_ylabel("Interocular acuity difference (logMAR)")

In [ ]:
np.corrcoef(has_motor['motorGABA'], has_motor['iadLogMAR'])

In [ ]:
st.pearsonr(has_motor['motorGABA'], has_motor['iadLogMAR'])